In [5]:
from data_preparation.utils import output_preprocessing_directory
import pandas as pd
from model.PrepareData import PrepareData
import re

In [21]:
from collections import Counter

def raw_chords_to_df(tunes, remove_root=False):
  tunes_chords = [item for tune in tunes for item in tune]

  if remove_root:
    tunes_chords = [re.sub('^[A-G][#b]?', '', chord) for chord in tunes_chords]

  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

# Read Data

In [22]:
#df = pd.read_csv(f'{output_preprocessing_directory}/03b_input_wordembedding_sections_full.csv', sep='\t')
prep = PrepareData('chordsFull', ngrams=[1])
df = prep.df_section['chords']
df.head()

Train Corpus: 3078
Test Corpus: 180


0    [CM7, Eb7, AbM7, B7, EM7, G7, Gm7, C7, FM7, Ab...
1    [Gm7, C7, Bm7, E7, AM7, C7, FM7, Bbm7, Eb7, Ab...
2    [Am7, Am7, Cm7, Cm7, EbM7, EbM7, Em7b5, A7(+#9...
3    [Am7, C#M7, Bm7b5, E7(+#9), Am7, C#M7, Bm7b5, ...
4    [C, Am7, Dm7, G7, C, Am7, Dm7, G7, C, Am7, Dm7...
Name: chords, dtype: object

### Prepare Overview for Full Chords, any Root

In [23]:
df_chords = raw_chords_to_df(df)
df_chords.head()

,chord,count
5,G7,4838
14,Dm7,3960
0,CM7,2590
32,C,2129
12,Am7,2013


### Prepare Overview for Full Chords, Roots removed

In [32]:
df_noroot = raw_chords_to_df(df, remove_root=True)

In [33]:
df_noroot.head(100)


,chord,count
1,7,13830
2,m7,9830
0,M7,4905
7,,3213
8,6,1657
5,7(+b9),1653
3,m7b5,1557
15,m,1462
11,dim7,775
10,m6,553
